<h1 style="text-align: center">机器学习导论习题六</h1>

<h2 style="text-align: center;">221300079, 王俊童, <a href="mailto:221300079@smail.nju.edu.cn">221300079@smail.nju.edu.cn</a></h2>

1. **10pts** **成功参赛**  成功提交私榜不低于 Baseline 1 的预测结果, <b><font color="red">队伍名称必须包含学号</font></b>. 比赛链接为 https://www.kaggle.com/t/34bc38e3ee15438fbbb40c8ef54adc3d, 比赛时间为 <b><font color="red">6月25日至7月10日</font></b>.
2. **25pts** **比赛排名** 预测结果在私榜不低于 Baseline 2 或者 Baseline 3 任一的前提下, 按照下式赋分: $10 + 15 \times \left( 1 - \frac{\textrm{你的排名} - 1}{\textrm{达到 baseline 的总人数}} \right)$. <b><font color="red">请确保<code>Kaggle.ipynb</code>能够复现你的结果.</font></b>

In [ ]:
# 用于记录每个单元格的运行时间

try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime

In [5]:
import os
import re
import glob
import time
import random
import datetime
import multiprocessing as mp
import numpy as np
import pandas as pd
import sklearn
import joblib
import torch
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

%matplotlib inline

matplotlib.rcParams['figure.dpi'] = 128
matplotlib.rcParams['figure.figsize'] = (8, 6)

time: 15 ms (started: 2024-06-26 15:26:52 +08:00)


In [6]:
# 固定随机数种子

GLOBAL_SEED = 0

def fix_seed(seed: int) -> None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

fix_seed(GLOBAL_SEED)

time: 47 ms (started: 2024-06-26 15:26:54 +08:00)


In [7]:
# 加载数据

df_train_x = pd.read_csv('kaggle-data/train_x.csv', index_col='id')
df_train_y = pd.read_csv('kaggle-data/train_y.csv', index_col='id')
df_test_x = pd.read_csv('kaggle-data/test_x.csv', index_col='id')
df_test_y_demo = pd.read_csv('kaggle-data/test_y_demo.csv', index_col='id')

train_x = df_train_x.values
train_y = df_train_y.values.reshape((-1,))
test_x = df_test_x.values

time: 3.55 s (started: 2024-06-26 15:26:56 +08:00)


In [8]:
def standardize(x, mu=None, sigma=None):
    if mu is None:
        mu = np.mean(x, axis=0)
    if sigma is None:
        sigma = np.std(x, axis=0) + 1e-3
    return (x - mu) / sigma, mu, sigma

train_x, mu, sigma = standardize(train_x)
test_x, _, _ = standardize(test_x, mu, sigma)

param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto', 0.1, 1, 10],
    'degree': [2, 3, 4] 
}

svc = SVC(random_state=GLOBAL_SEED)
grid_search = GridSearchCV(estimator=svc, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(train_x, train_y)
print("Best parameters found: ", grid_search.best_params_)
best_svc = grid_search.best_estimator_
test_y = best_svc.predict(test_x)
df = pd.read_csv('kaggle-data/test_y_demo.csv', index_col='id')
df['y'] = test_y  
df.to_csv('test_y.csv')

time: 125 ms (started: 2024-06-26 15:27:11 +08:00)


# 致谢

<h2>允许与其他同样未完成作业的同学讨论作业的内容, 但需在此注明并加以致谢; 如在作业过程中, 参考了互联网上的资料或大语言模型的生成结果, 且对完成作业有帮助的, 亦需注明并致谢.</h2>